# Baseline Model

## Table of Contents

1. [Model Choice](#model-choice)
2. [Feature Selection](#feature-selection)
3. [Implementation](#implementation)
4. [Evaluation](#evaluation)


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
# Import your chosen baseline model
# Example: from sklearn.linear_model import LogisticRegression

# load the training data
df_raw_train = pd.read_csv('../data/train.csv')


## Model Choice

[Explain why you've chosen a particular model as the baseline. This could be a simple statistical model or a basic machine learning model. Justify your choice.]

As a baseline model a very simple decison tree is implemented. The decision tree is well interpretable, making it possible to check for errors in the dataset, model and evaluationn pipeline. 




## Feature Selection

[Indicate which features from the dataset you will be using for the baseline model, and justify your selection.]

Only Heart Rate (HR), Body Temperature (Temp), Respiratory Rate (Resp) and Systolic Blood Pressure (SBP) will be used for prediction as they are readily available and already routinely used in clinical practice (See qSOFA or SIRS [Please Note: WBC altough part of SIRS criteria is not taken into account as only available in <10% of data points]) 

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold

DATA_DIR = "/teamspace/studios/this_studio/detecting_Sepsis/data/!LIGHTGBM_DATA"
df = pd.read_csv(f"{DATA_DIR}/train_fit.csv")

# cleanup
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

TARGET = "SepsisLabel"
GROUP  = "Patient_ID"

# optional stable ordering
sort_cols = [c for c in [GROUP, "ICULOS", "Hour"] if c in df.columns]
df = df.sort_values(sort_cols).reset_index(drop=True)

# patient-level stratification label
patient_y = df.groupby(GROUP)[TARGET].max().astype(int)
patient_ids = patient_y.index.to_numpy()
patient_labels = patient_y.values

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_metrics = []
for fold, (tr_idx, va_idx) in enumerate(skf.split(patient_ids, patient_labels), 1):
    tr_pids = set(patient_ids[tr_idx])
    va_pids = set(patient_ids[va_idx])

    tr = df[df[GROUP].isin(tr_pids)]
    va = df[df[GROUP].isin(va_pids)]

    y_tr = tr[TARGET].astype(int).to_numpy()
    y_va = va[TARGET].astype(int).to_numpy()

    X_tr = tr.drop(columns=[TARGET, GROUP])
    X_va = va.drop(columns=[TARGET, GROUP])

    # imbalance weight (row-level)
    pos = (y_tr == 1).sum()
    neg = (y_tr == 0).sum()
    spw = neg / max(pos, 1)

    model = lgb.LGBMClassifier(
        objective="binary",
        metric ="auc",
        n_estimators=4000,
        learning_rate=0.03,
        num_leaves=64,
        min_child_samples=50,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        n_jobs=-1,
        random_state=42,
        scale_pos_weight=spw,
    )

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(100)]
    )

    p_va = model.predict_proba(X_va)[:, 1]
    auroc = roc_auc_score(y_va, p_va)
    auprc = average_precision_score(y_va, p_va)

    fold_metrics.append((auroc, auprc))
    print(f"Fold {fold}: AUROC={auroc:.4f}  AUPRC={auprc:.4f}")

m = np.array(fold_metrics)
print(f"\nMean AUROC={m[:,0].mean():.4f} ± {m[:,0].std():.4f}")
print(f"Mean AUPRC={m[:,1].mean():.4f} ± {m[:,1].std():.4f}")


[LightGBM] [Info] Number of positive: 17004, number of negative: 927794
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.119659 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6987
[LightGBM] [Info] Number of data points in the train set: 944798, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.017997 -> initscore=-3.999361
[LightGBM] [Info] Start training from score -3.999361
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.794879
[200]	valid_0's auc: 0.794406
[300]	valid_0's auc: 0.793232
Early stopping, best iteration is:
[130]	valid_0's auc: 0.795586
Fold 1: AUROC=0.7956  AUPRC=0.0852
[LightGBM] [Info] Number of positive: 16985, number of negative: 927378
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121366 seconds.
You can set 

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb

DATA_DIR = "/teamspace/studios/this_studio/detecting_Sepsis/data/!LIGHTGBM_DATA"
TRAIN_PATH = f"{DATA_DIR}/train_fit.csv"

TARGET = "SepsisLabel"
GROUP  = "Patient_ID"
BEST_N = 122  # median best_iteration from your 5-fold CV

# -------------------
# Load + cleanup
# -------------------
df = pd.read_csv(TRAIN_PATH)

if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# stable order (optional)
sort_cols = [c for c in [GROUP, "ICULOS", "Hour"] if c in df.columns]
df = df.sort_values(sort_cols).reset_index(drop=True)

# -------------------
# Features / target
# -------------------
y_full = df[TARGET].astype(int).to_numpy()
X_full = df.drop(columns=[TARGET, GROUP])

# class imbalance weight on FULL train_fit
pos = int((y_full == 1).sum())
neg = int((y_full == 0).sum())
scale_pos_weight = neg / max(pos, 1)
print("Full train_fit rows:", len(df), "pos:", pos, "neg:", neg, "scale_pos_weight:", scale_pos_weight)

# -------------------
# Final model (no early stopping)
# -------------------
final_model = lgb.LGBMClassifier(
    objective="binary",
    metric="auc",
    n_estimators=BEST_N,        # <-- fixed based on CV
    learning_rate=0.03,
    num_leaves=64,
    min_child_samples=50,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    force_row_wise=True,       # avoid the overhead message
)

final_model.fit(X_full, y_full)

# -------------------
# Save the model
# -------------------
MODEL_PATH = os.path.join(DATA_DIR, f"lgbm_final_{BEST_N}trees.txt")
final_model.booster_.save_model(MODEL_PATH)
print("Saved final model to:", MODEL_PATH)

# Optional: save feature names (useful later)
FEAT_PATH = os.path.join(DATA_DIR, "lgbm_feature_names.txt")
with open(FEAT_PATH, "w") as f:
    for c in X_full.columns:
        f.write(c + "\n")
print("Saved feature names to:", FEAT_PATH)


Full train_fit rows: 1180166 pos: 21247 neg: 1158919 scale_pos_weight: 54.54506518567327
[LightGBM] [Info] Number of positive: 21247, number of negative: 1158919
[LightGBM] [Info] Total Bins 7021
[LightGBM] [Info] Number of data points in the train set: 1180166, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.018003 -> initscore=-3.999027
[LightGBM] [Info] Start training from score -3.999027
Saved final model to: /teamspace/studios/this_studio/detecting_Sepsis/data/!LIGHTGBM_DATA/lgbm_final_122trees.txt
Saved feature names to: /teamspace/studios/this_studio/detecting_Sepsis/data/!LIGHTGBM_DATA/lgbm_feature_names.txt


In [4]:
# ============================================================
# LightGBM -> Official evaluator threshold sweep (EXACT)
# Fixes PSV/train_fit feature mismatch (Hour vs ICULOS, etc.)
# Writes predictions to your folder and calls evaluate_sepsis_score.py
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd
import lightgbm as lgb
import importlib.util


# -----------------------------
# Paths (edit if needed)
# -----------------------------
EVAL_SCRIPT = "/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/evaluate_sepsis_score.py"

DATA_DIR = "/teamspace/studios/this_studio/detecting_Sepsis/data/!LIGHTGBM_DATA"
LABEL_DIR_THRESH = f"{DATA_DIR}/PSV_Patients_THRESH"
LABEL_DIR_TEST   = f"{DATA_DIR}/PSV_Patients_TEST"

PRED_ROOT = "/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/LIGHTGBM"
PRED_DIR_THRESH_WORK = f"{PRED_ROOT}/THRESH_WORK"   # overwritten each threshold
PRED_DIR_TEST_FINAL  = f"{PRED_ROOT}/TEST_FINAL"

MODEL_PATH = f"{DATA_DIR}/lgbm_final_122trees.txt"
FEATURES_PATH = f"{DATA_DIR}/lgbm_feature_names.txt"


# -----------------------------
# Load official evaluator (exact)
# -----------------------------
def load_official_evaluator(eval_script_path: str):
    spec = importlib.util.spec_from_file_location("eval_sepsis", eval_script_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod


# -----------------------------
# Model + features
# -----------------------------
def load_booster(model_path: str) -> lgb.Booster:
    return lgb.Booster(model_file=model_path)

def load_feature_list(path: str) -> list[str]:
    with open(path, "r") as f:
        feats = [line.strip() for line in f if line.strip()]
    return feats


# -----------------------------
# IO helpers
# -----------------------------
def list_patient_files(psv_dir: str) -> list[Path]:
    return sorted(Path(psv_dir).glob("*.psv"))

def ensure_empty_dir(dir_path: str):
    d = Path(dir_path)
    d.mkdir(parents=True, exist_ok=True)
    for f in d.glob("*.psv"):
        f.unlink()

def write_prediction_file(out_path: Path, prob: np.ndarray, threshold: float):
    pred = (prob >= threshold).astype(int)
    out_df = pd.DataFrame({
        "PredictedProbability": prob,
        "PredictedLabel": pred
    })
    out_df.to_csv(out_path, sep="|", index=False)


# -----------------------------
# Feature alignment: PSV -> model features
# -----------------------------
def prepare_features_from_psv(df: pd.DataFrame, feature_cols: list[str]) -> pd.DataFrame:
    """
    Make a PSV dataframe compatible with the model's expected features:
      - Drop SepsisLabel if present (never use as feature)
      - If model expects Hour but PSV doesn't have it, derive Hour = ICULOS - 1
      - If model expects ICULOS but PSV doesn't have it, derive ICULOS = Hour + 1
      - Add any other missing expected columns as NaN
      - Return columns in EXACT training order
    """
    df = df.drop(columns=["SepsisLabel"], errors="ignore")

    # Derive Hour / ICULOS if needed
    if "Hour" in feature_cols and "Hour" not in df.columns:
        if "ICULOS" in df.columns:
            df["Hour"] = df["ICULOS"] - 1
        else:
            df["Hour"] = np.nan

    if "ICULOS" in feature_cols and "ICULOS" not in df.columns:
        if "Hour" in df.columns:
            df["ICULOS"] = df["Hour"] + 1
        else:
            df["ICULOS"] = np.nan

    # Add missing model features as NaN
    for c in feature_cols:
        if c not in df.columns:
            df[c] = np.nan

    return df[feature_cols]

def read_features_from_patient_file(psv_path: Path, feature_cols: list[str]) -> pd.DataFrame:
    df = pd.read_csv(psv_path, sep="|")
    return prepare_features_from_psv(df, feature_cols)


# -----------------------------
# Core: cache probabilities once (per split)
# -----------------------------
def cache_probabilities(label_dir: str, booster: lgb.Booster, feature_cols: list[str]) -> dict[str, np.ndarray]:
    probs = {}
    files = list_patient_files(label_dir)
    if not files:
        raise FileNotFoundError(f"No .psv files found in: {label_dir}")

    for fp in files:
        X = read_features_from_patient_file(fp, feature_cols)
        probs[fp.name] = booster.predict(X)  # prob per row
    return probs


# -----------------------------
# Core: materialize predictions for a threshold
# -----------------------------
def materialize_predictions(probs: dict[str, np.ndarray], out_dir: str, threshold: float):
    ensure_empty_dir(out_dir)
    out = Path(out_dir)
    for fname, p in probs.items():
        write_prediction_file(out / fname, p, threshold)


# -----------------------------
# Evaluate (official)
# -----------------------------
def official_eval(evaluator_module, label_dir: str, pred_dir: str):
    # returns: auroc, auprc, accuracy, f_measure, normalized_observed_utility
    return evaluator_module.evaluate_sepsis_score(label_dir, pred_dir)



In [6]:

# ============================================================
# Run threshold sweep on THRESH (exact official scoring)
# ============================================================
Path(PRED_ROOT).mkdir(parents=True, exist_ok=True)

evaluator = load_official_evaluator(EVAL_SCRIPT)
booster = load_booster(MODEL_PATH)
feature_cols = load_feature_list(FEATURES_PATH)

# Cache probabilities ONCE for THRESH (fast; no re-predicting for every threshold)
probs_thresh = cache_probabilities(LABEL_DIR_THRESH, booster, feature_cols)

thresholds = np.round(np.arange(0.01, 0.91, 0.01), 2) 
rows = []
best = None

for t in thresholds:
    t = float(t)
    materialize_predictions(probs_thresh, PRED_DIR_THRESH_WORK, t)
    auroc, auprc, acc, f1, util = official_eval(evaluator, LABEL_DIR_THRESH, PRED_DIR_THRESH_WORK)

    rows.append((t, util, auroc, auprc, acc, f1))

    if best is None or util > best[1]:
        best = (t, util, auroc, auprc, acc, f1)
        print(f"NEW BEST t={t:.3f} | Utility={util:.5f} | AUROC={auroc:.4f} AUPRC={auprc:.4f}")

results = pd.DataFrame(rows, columns=["threshold","utility","auroc","auprc","accuracy","f1"])
out_csv = f"{PRED_ROOT}/thresh_sweep_results.csv"
results.to_csv(out_csv, index=False)

print("\nBest threshold found:")
print(best)
print("Saved sweep results to:", out_csv)



NEW BEST t=0.010 | Utility=-0.60402 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.020 | Utility=-0.60313 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.030 | Utility=-0.59913 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.040 | Utility=-0.58524 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.050 | Utility=-0.56408 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.060 | Utility=-0.53880 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.070 | Utility=-0.50597 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.080 | Utility=-0.47070 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.090 | Utility=-0.43509 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.100 | Utility=-0.40085 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.110 | Utility=-0.36357 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.120 | Utility=-0.32933 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.130 | Utility=-0.29589 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.140 | Utility=-0.26317 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.150 | Utility=-0.22864 | AUROC=0.7806 AUPRC=0.0885
NEW BEST t=0.160 | Utility=-0.19593 | AU

In [7]:

# ============================================================
# (Optional) Write FINAL TEST predictions using best threshold
# ============================================================
best_t = float(best[0])

probs_test = cache_probabilities(LABEL_DIR_TEST, booster, feature_cols)
materialize_predictions(probs_test, PRED_DIR_TEST_FINAL, best_t)

print("\nWrote TEST predictions to:", PRED_DIR_TEST_FINAL)
print("Best threshold used:", best_t)

# If your TEST folder has labels and you want to score locally (optional):
# auroc, auprc, acc, f1, util = official_eval(evaluator, LABEL_DIR_TEST, PRED_DIR_TEST_FINAL)
# print("TEST (official):", {"auroc": auroc, "auprc": auprc, "acc": acc, "f1": f1, "utility": util})



Wrote TEST predictions to: /teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/LIGHTGBM/TEST_FINAL
Best threshold used: 0.55


In [8]:
auroc, auprc, acc, f1, util = official_eval(evaluator, LABEL_DIR_TEST, PRED_DIR_TEST_FINAL)

print("TEST (official):")
print(f"AUROC    = {auroc:.6f}")
print(f"AUPRC    = {auprc:.6f}")
print(f"Accuracy = {acc:.6f}")
print(f"F1       = {f1:.6f}")
print(f"Utility  = {util:.6f}")


TEST (official):
AUROC    = 0.817455
AUPRC    = 0.110526
Accuracy = 0.848159
F1       = 0.125706
Utility  = 0.383143


## Implementation

[Implement your baseline model here.]



In [ ]:
# Initialize and train the baseline model


## Evaluation

[Clearly state what metrics you will use to evaluate the model's performance. These metrics will serve as a starting point for evaluating more complex models later on.]



In [ ]:
# Evaluate the baseline model
# Example for a classification problem
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)

# For a regression problem, you might use:
# mse = mean_squared_error(y_test, y_pred)

# Your evaluation code here
